In [9]:
import copy
import time

import numpy as np
import torch
from sklearn import model_selection
import matplotlib.pyplot as plt
import math
import Sampling_Methods

from KnapsackSolver import compute_profit_knapsack_single_benchmark, get_opt_params_knapsack
from Sampling_Methods import find_transition_intervals, Sampler, EXHAUSTIVE, MID_TRANSITION_POINT_SELECTION, \
    DIVIDE_AND_CONQUER, EDGE_TRANSITION_POINT_SELECTION
from Solver import get_optimization_objective_for_samples, compute_objective_value_single_benchmarks
from linear_nn import linear_regression
from relu_ppo import relu_ppo
import applied_tests
np.random.seed(0)


In [17]:
benchmark_size = 50
benchmark_numbers = 100
dataset = applied_tests.init_polynomial_set(benchmark_size, benchmark_numbers)
X_train, X_test, y_train, y_test = model_selection.train_test_split(dataset.get('x').T, dataset.get('y').T,
                                                                    test_size=0.2, shuffle=True)
layer_params_relu = [1, 10, 1]
my_relu_regression = relu_ppo(batch_size = 16,
                              layer_params=layer_params_relu, max_epoch=5,
                              opt_params=get_opt_params_knapsack(capacity=10))
my_relu_regression.fit_nn(X_train, y_train)

layer_params_linear = [1, 1]
my_linear_regression = linear_regression(N=10, layer_params=layer_params_linear, max_epoch=20)
my_linear_regression.fit_nn(X_train, y_train)

e:0 i:0/4000 loss:1.5887641906738281
e:1 i:0/4000 loss:0.8223134875297546
e:2 i:0/4000 loss:0.6204252243041992
e:3 i:0/4000 loss:0.7147856950759888
e:4 i:0/4000 loss:0.6106580495834351
e:0 i:0/4000 loss:1.3789480924606323
e:1 i:0/4000 loss:1.2790515422821045
e:2 i:0/4000 loss:0.9253883361816406
e:3 i:0/4000 loss:0.8883631825447083
e:4 i:0/4000 loss:0.9295905828475952
e:5 i:0/4000 loss:0.9167416095733643
e:6 i:0/4000 loss:0.586035966873169
e:7 i:0/4000 loss:0.8827596306800842
e:8 i:0/4000 loss:0.6365388631820679
e:9 i:0/4000 loss:0.6935698986053467
e:10 i:0/4000 loss:0.7265133857727051
e:11 i:0/4000 loss:0.6330640912055969
e:12 i:0/4000 loss:0.5461883544921875
e:13 i:0/4000 loss:0.5971035361289978
e:14 i:0/4000 loss:0.6591071486473083
e:15 i:0/4000 loss:0.7528563141822815
e:16 i:0/4000 loss:0.930079460144043
e:17 i:0/4000 loss:0.7529701590538025
e:18 i:0/4000 loss:0.827538251876831
e:19 i:0/4000 loss:0.7776552438735962


In [20]:
def relu_dnl(self, x, y, weights, benchmark_numbers, benchmark_size, sampling_method=Sampling_Methods.DIVIDE_AND_CONQUER, max_step_size_magnitude=0,
                min_step_size_magnitude=-1,
                transition_point_selection=Sampling_Methods.MID_TRANSITION_POINT_SELECTION):

        dnc_sampler = Sampling_Methods.Sampler(sampling_method=sampling_method,
                                               max_step_size_magnitude=max_step_size_magnitude,
                                               min_step_size_magnitude=min_step_size_magnitude,
                                               transition_point_selection=transition_point_selection)
        temp_model = copy.deepcopy(self)

        number_of_batches = int(benchmark_numbers / self.batch_size)


        for t in range(self.epoch_number):
            # UNCOMMENT BELOW FOR RANDOM SHUFFLE BATCHES
            # permutations = torch.randperm(number_of_benchmarks).tolist()
            # bELOW IS FOR ORDERED BATCHES
            permutations = [x for x in range(number_of_batches)]
            for i in range(0, number_of_batches, self.batch_size):
                indices = [x for y in range(permutations[i], permutations[i] + self.batch_size) for x in
                           range(y * benchmark_size, (y + 1) * benchmark_size)]
                batch_x, batch_y, batch_weights = x[indices], y[indices], weights[indices]
                for benchmark_x, benchmark_y, benchmark_weight in zip(batch_x, batch_y, batch_weights):
                    for layer_no in range(2):
                        if layer_no == 0:
                            current_temp_layer = temp_model.fc1
                            current_original_layer = self.fc1



                        else:
                            current_temp_layer = temp_model.fc2
                            current_original_layer = self.fc2

                        for param_ind in [(i, j) for i in range(current_original_layer.weight.size()[0]) for j in
                                          range(current_original_layer.weight.size()[1])]:

                            dnc_points, dnc_intervals, __, dnc_POVS, dnc_TOVS = dnc_sampler.get_transition_points(
                                temp_model,
                                current_temp_layer,
                                benchmark_x,
                                benchmark_y, weights,
                                param_ind)

                            with torch.no_grad():
                                current_temp_layer.weight[param_ind] = current_original_layer.weight[param_ind]

In [23]:

weights = np.array([np.ones(benchmark_size) for i in range(benchmark_numbers)]).T
relu_dnl(my_relu_regression, X_train, y_train, weights, benchmark_numbers = benchmark_numbers, benchmark_size = benchmark_size)


IndexError: index 50 is out of bounds for axis 0 with size 50